<h1 style="text-align: center;">Création d'un chatbot avec MemN2N utilisant le mécanisme de l’Attention</h1>


# I) Introduction

<p>Dans ce tutoriel, nous allons explorer comment créer un chatbot qui peut répondre à des questions en utilisant un modèle de réseau de mémoire end-to-end (MemN2N) avec le mécanisme d'attention. Le modèle sera entraîné sur un ensemble de données qui contient des histoires, des questions et des réponses « oui » ou « non » associées. Nous allons également expliquer comment prétraiter les données en séquences de mots encodées sous forme de nombres pour être traitées par le modèle.</p>

<p>Le mécanisme d'attention est un composant clé de notre modèle de chatbot. Il permet au modèle de se concentrer sur les parties importantes des données d'entrée tout en ignorant les parties moins importantes. Nous allons discuter de la manière dont le mécanisme d'attention fonctionne et de son rôle dans la création d'un chatbot efficace.</p>

<p>Nous allons également plonger dans MemN2N, un modèle de réseau de mémoire end-to-end qui utilise une approche d'apprentissage en mémoire pour résoudre des tâches de question-réponse. Nous expliquerons comment utiliser ce modèle pour construire notre chatbot.</p>

<p>Pour ce faire, nous allons utiliser l'ensemble de données Babi de Facebook Research pour entraîner notre modèle. Cet ensemble de données comprend plusieurs histoires courtes qui contiennent des questions et des réponses. Nous allons prétraiter ces données pour les utiliser avec notre modèle de chatbot.</p>







# II) Prétraitement des données

<p>On importe les librairies nécéssaires.</p>

In [61]:
# Importatation des librairies nécéssaires
import pickle
import numpy as np

from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer


from keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding
from keras.layers import Input, Activation, Dense, Permute, Dropout
from keras.layers import add, dot, concatenate
from keras.layers import LSTM

<p>Les lignes de code suivantes utilisent le module Python pickle pour charger des données prétraitées à partir de fichiers de texte dans les variables <code>train_data</code> et <code>test_data</code>.</p>

<p>La première ligne ouvre le fichier "train_qa.txt" en mode binaire ("rb") et utilise le module pickle pour désérialiser les données dans le fichier. Les données désérialisées sont stockées dans la variable <code>train_data</code>. La deuxième ligne suit le même processus pour le fichier "test_qa.txt", stockant les données désérialisées dans la variable <code>test_data</code>.</p>


In [62]:
with open("train_qa.txt", "rb") as fp:   
    train_data =  pickle.load(fp)
    
with open("test_qa.txt", "rb") as fp:  
    test_data =  pickle.load(fp)

<p>Ici on crée un ensemble (<code>set</code>) qui contiendra tous les mots présents dans le dataset de données utilisé.</p>
<ol>
<li>On crée un ensemble vide appelé "<code>vocab</code>".</li>
<li>On combine les données d'entraînement et de test en une seule variable appelée "<code>all_data</code>".</li>
<li>Pour chaque histoire (<code>story</code>), question et réponse (<code>answer</code>) dans "<code>all_data</code>", on ajoute tous les mots uniques de l'histoire et de la question à l'ensemble "<code>vocab</code>".</li>
<li>On ajoute les mots "<code>no</code>" et "<code>yes</code>" à l'ensemble "<code>vocab</code>" </li>
<li>On calcule la longueur de l'ensemble "<code>vocab</code>" en ajoutant 1 pour inclure un index réservé pour les mots inconnus.</li>
</ol>
<p> Qu'est-ce qu'un "<code>set</code>" et "<code>union</code>" :</p>
<ul>
<li>Un "<code>set</code>" en Python est une structure de données qui stocke des éléments uniques et non ordonnés. Il est similaire à une liste ou un tuple, mais avec l'ajout que chaque élément est unique, c'est-à-dire qu'il n'y a pas de doublons dans un ensemble.</li>
<li>La méthode "<code>union</code>" est utilisée pour combiner deux ensembles en un seul ensemble qui contient tous les éléments uniques de ces ensembles. En d'autres termes, la méthode "<code>union</code>" permet d'ajouter des éléments à un ensemble sans créer de doublons.</li>
</ul>


In [63]:
# On crée un ensemble (set) qui contiendra tout les mots dans notre dataset
vocab = set()
all_data = test_data + train_data

for story, question , answer in all_data:

    vocab = vocab.union(set(story))
    vocab = vocab.union(set(question))
    
vocab.add('no')
vocab.add('yes')   
vocab_len = len(vocab) + 1

<p>Ici, on calcule la longueur maximale des histoires "<code>story</code>"  et des questions "<code>questions</code>"dans le dataset de données "<code>all_data</code>".</p>

In [ ]:
max_story_len = max([len(data[0]) for data in all_data])

max_question_len = max([len(data[1]) for data in all_data])

<p> On convertit chaque séquence de mots en une séquence d'entiers à l'aide de la classe "<code>Tokenizer</code>" et de la méthode "<code>texts_to_sequences</code>". Les données d'entraînement (histoires, questions et réponses) sont préparées pour l'entraînement du modèle de chatbot en vue de construire un modèle capable de comprendre les séquences de mots et de prédire des réponses adéquates.</p>
<ol>
<li>On crée une instance de la classe "<code>Tokenizer</code>" du module "tensorflow.keras.preprocessing.text" pour convertir chaque séquence de mots en une séquence d'entiers. On utilise l'argument "filters" avec une valeur vide pour indiquer au tokenizer de ne pas supprimer les caractères spéciaux par défaut.<br>
<code>tokenizer = Tokenizer(filters=[])</code></li>
<li>On utilise la méthode "<code>fit_on_texts</code>" du tokenizer pour adapter les données d'entraînement ("vocab") à notre tokenizer. Cette méthode transforme chaque mot unique en un index numérique.<br>
<code>tokenizer.fit_on_texts(vocab)</code></li>
<li>On crée trois listes vides : "<code>train_story_text</code>", "<code>train_question_text</code>" et "<code>train_answers</code>".<br>

<li>On parcourt chaque donnée d'entraînement "<code>train_data</code>" et on extrait l'histoire, la question et la réponse de chaque donnée. On ajoute chaque histoire dans la liste "<code>train_story_text</code>", chaque question dans la liste "<code>train_question_text</code>", et chaque réponse dans la liste "<code>train_answers</code>".<br>


<li>On utilise la méthode "<code>texts_to_sequences</code>" du tokenizer pour convertir chaque élément de la liste "<code>train_story_text</code>" en une séquence d'entiers correspondante. Cette méthode effectue les tâches suivantes : tokenization des mots, remplacement de chaque mot par son index dans le dictionnaire, et padding/troncature de chaque séquence pour qu'elles aient la même longueur.<br>
<code>train_story_seq = tokenizer.texts_to_sequences(train_story_text)</code></li>
</ol>


In [ ]:

tokenizer = Tokenizer(filters=[])
tokenizer.fit_on_texts(vocab)

train_story_text = []
train_question_text = []
train_answers = []

for story,question,answer in train_data:
    train_story_text.append(story)
    train_question_text.append(question)
    
train_story_seq = tokenizer.texts_to_sequences(train_story_text)


<p>On va créer une fonction qui permet de transformer une phrase en une séquence de nombres en remplaçant chaque mot par son index dans le dictionnaire d'index de mots <code>tokenizer.word_index</code> , qui est fourni en entrée de la fonction. La fonction prend également en entrée deux paramètres qui correspondent à la longueur maximale de l'histoire et de la question. Ces paramètres sont utilisés pour ajouter des zéros à la fin de la séquence si elle est plus courte que la longueur maximale spécifiée. La fonction retourne donc une séquence de nombres (avec des zéros ajoutés si nécessaire) pour chaque phrase (histoire ou question) dans les données d'entrée. </p>


In [31]:
def stories_vectorization(data, word_index=tokenizer.word_index, max_story_len=max_story_len,max_question_len=max_question_len):    
    
    X = []
    Xq = []
    Y = []
    
    
    for story, query, answer in data:
        
       
        x = [word_index[word.lower()] for word in story] 
        xq = [word_index[word.lower()] for word in query]
        y = np.zeros(len(word_index) + 1)
        y[word_index[answer]] = 1
        
        X.append(x)
        Xq.append(xq)
        Y.append(y)
        
    
    return (pad_sequences(X, maxlen=max_story_len),pad_sequences(Xq, maxlen=max_question_len), np.array(Y))

In [32]:
inputs_train, queries_train, answers_train = stories_vectorization(train_data)
inputs_test, queries_test, answers_test = stories_vectorization(test_data)
inputs_test
tokenizer.word_index['yes']
tokenizer.word_index['no']

array([[ 0,  0,  0, ..., 19, 17, 15],
       [ 0,  0,  0, ..., 19,  5, 15],
       [ 0,  0,  0, ..., 19,  5, 15],
       ...,
       [ 0,  0,  0, ..., 19, 27, 15],
       [ 0,  0,  0, ..., 19,  5, 15],
       [ 0,  0,  0, ..., 27, 20, 15]])

# III) Entrainement du modèle

<p> Pour créer notre modèle, on va utiliser l'architecture <code>MemN2N</code> pour entraîner un chatbot. MemN2N est une architecture de réseau de neurones qui utilise une approche de mémoire à court terme pour répondre aux questions posées en utilisant une histoire donnée. Pour en savoir plus sur cette architecture, vous pouvez consulter l'article original : <a href="http://arxiv.org/abs/1503.08895">End-To-End Memory Networks</a>. </p>

<p>Ces deux lignes de code vont créer deux tenseurs d'entrée pour le modèle.</p>
<p> <code>Input((max_story_len,))</code> ve créer un tenseur d'entrée pour les histoires, qui est une séquence d'entiers de longueur <code>max_story_len</code>.</p>
<p><code>Input((max_question_len,))</code> va créer un tenseur d'entrée pour les questions, qui est une séquence d'entiers de longueur <code>max_question_len</code>.</p>
<p>Ces tenseurs d'entrée sont nécessaires pour entraîner le modèle et sont utilisés pour fournir les données d'entrée (histoires et questions) au réseau de neurones lors de l'entraînement et de l'évaluation. Les tenseurs d'entrée définis par ces lignes de code sont passés en tant qu'arguments à la méthode <code>fit()</code> lors de l'entraînement du modèle.</p>

In [34]:
input_sequence = Input((max_story_len,))
question = Input((max_question_len,))



<p>Dans l'architecture MemN2N, l'idée est de combiner l'histoire (story) et la question pour prédire la réponse. L'histoire peut être considérée comme une séquence de mots qui peut être encodée en une séquence de vecteurs denses. Cependant, le traitement de la question est différent car elle est plus courte que l'histoire et a une signification différente.</p>

<p>
    Dans l'architecture MemN2N, deux encodeurs sont utilisés pour représenter l'histoire et la question. L'encodeur <code>input_encoder_m</code> est utilisé pour encoder la séquence de l'histoire (<code>input_sequence</code>) en une séquence de vecteurs denses de taille fixe. L'encodeur <code>input_encoder_c</code> est également utilisé pour encoder la séquence de l'histoire (<code>input_sequence</code>), mais cette fois-ci pour encoder la séquence en une séquence de vecteurs denses de taille <code>max_question_len</code>. Cela permet de tenir compte de la question dans le calcul de la réponse.

L'utilisation de ces deux encodeurs différents pour représenter l'histoire et la question permet d'avoir deux représentations distinctes pour chaque entrée, ce qui permet au modèle de tenir compte de la question lors de la prédiction de la réponse.
  </p>




<p>Pour résumer,dans l'architecture MemN2N, il y a trois encodeurs qui sont utilisés pour représenter l'histoire, la question et la correspondance entre l'histoire et la question.</p>
<ul>
  <li><p>L'encodeur <code>input_encoder_m</code> est utilisé pour encoder la séquence de l'histoire (<code>input_sequence</code>) en une séquence de vecteurs denses de taille fixe. L'argument <code>output_dim</code> de cet encodeur est défini sur 64 dimensions pour créer une représentation dense de l'histoire.</p></li>
  <li><p>L'encodeur <code>input_encoder_c</code> est utilisé pour encoder la séquence de l'histoire (<code>input_sequence</code>) en une séquence de vecteurs denses de taille <code>max_question_len</code>. L'argument <code>output_dim</code> de cet encodeur est défini sur <code>max_question_len</code> dimensions pour créer une représentation dense qui correspond à la question. Cela permet de tenir compte de la question dans le calcul de la réponse.</p></li>
  <li><p>L'encodeur <code>question_encoder</code> est utilisé pour encoder la séquence de la question en une séquence de vecteurs denses. L'argument <code>output_dim</code> de cet encodeur est également défini sur 64 dimensions pour créer une représentation dense de la question. Cette représentation sera utilisée pour calculer la correspondance entre l'histoire et la question.</p></li>
</ul>
<p><b>Les trois encodeurs permettent de créer des représentations distinctes pour l'histoire, la question et leur correspondance, ce qui permet au modèle de tenir compte de la question lors de la prédiction de la réponse</b>.</p>


In [ ]:

input_encoder_m = Sequential()
input_encoder_m.add(Embedding(input_dim=vocab_size,output_dim=64))
input_encoder_m.add(Dropout(0.3))



input_encoder_c = Sequential()
input_encoder_c.add(Embedding(input_dim=vocab_size,output_dim=max_question_len))
input_encoder_c.add(Dropout(0.3))


question_encoder = Sequential()
question_encoder.add(Embedding(input_dim=vocab_size,
                               output_dim=64,
                               input_length=max_question_len))
question_encoder.add(Dropout(0.3))

<ul>
    <li><code>input_encoded_m</code>: la sortie de l'encodeur <code>input_encoder_m</code>. Cette variable représente l'histoire encodée en une séquence de vecteurs denses de taille fixe.</li>
    <li><code>input_encoded_c</code>: la sortie de l'encodeur <code>input_encoder_c</code>. Cette variable représente l'histoire encodée en une séquence de vecteurs denses de taille <code>max_question_len</code>.</li>
    <li><code>question_encoded</code>: la sortie de l'encodeur <code>question_encoder</code>. Cette variable représente la question encodée en une séquence de vecteurs denses de taille fixe.</li>
    <li><code>match</code>: la matrice de correspondance entre l'histoire et la question. Elle est calculée en prenant le produit scalaire entre les encodages de l'histoire et de la question. 
    <li><code>response</code>: la matrice résultante de l'addition de la matrice de correspondance et de l'encodeur <code>input_encoder_c</code>.)</li>
    <li><code>answer</code>: la concaténation de la matrice de correspondance et de l'encodeur <code>question_encoder</code>..</li>

</ul>

In [ ]:
# encode input sequence and questions (which are indices)
# to sequences of dense vectors
input_encoded_m = input_encoder_m(input_sequence)
input_encoded_c = input_encoder_c(input_sequence)
question_encoded = question_encoder(question)

# shape: `(samples, story_maxlen, query_maxlen)`
match = dot([input_encoded_m, question_encoded], axes=(2, 2))
match = Activation('softmax')(match)




# add the match matrix with the second input vector sequence
response = add([match, input_encoded_c])  # (samples, story_maxlen, query_maxlen)
response = Permute((2, 1))(response)  # (samples, query_maxlen, story_maxlen)

# concatenate the match matrix with the question vector sequence
answer = concatenate([response, question_encoded])

# Regularization with Dropout
answer = Dropout(0.5)(answer)
answer = Dense(vocab_size)(answer)  # (samples, vocab_size)

# we output a probability distribution over the vocabulary
answer = Activation('softmax')(answer)


<p> On entraine le modèle </p>

In [45]:
model = Model([input_sequence, question], answer)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])
history = model.fit([inputs_train, queries_train], answers_train,batch_size=32,epochs=120,validation_data=([inputs_test, queries_test], answers_test))

Epoch 1/120
313/313 [==============================] - 4s 7ms/step - loss: 0.8935 - accuracy: 0.4949 - val_loss: 0.6945 - val_accuracy: 0.5030
Epoch 2/120
313/313 [==============================] - 2s 6ms/step - loss: 0.7037 - accuracy: 0.5060 - val_loss: 0.6934 - val_accuracy: 0.5130
Epoch 3/120
313/313 [==============================] - 2s 7ms/step - loss: 0.6987 - accuracy: 0.4922 - val_loss: 0.6950 - val_accuracy: 0.5030
Epoch 4/120
313/313 [==============================] - 3s 10ms/step - loss: 0.6966 - accuracy: 0.5028 - val_loss: 0.6940 - val_accuracy: 0.5030
Epoch 5/120
313/313 [==============================] - 3s 9ms/step - loss: 0.6964 - accuracy: 0.4919 - val_loss: 0.6938 - val_accuracy: 0.5030
Epoch 6/120
313/313 [==============================] - 3s 10ms/step - loss: 0.6958 - accuracy: 0.4946 - val_loss: 0.6932 - val_accuracy: 0.4980
Epoch 7/120
313/313 [==============================] - 3s 11ms/step - loss: 0.6954 - accuracy: 0.5031 - val_loss: 0.6932 - val_accuracy: 0.5

313/313 [==============================] - 4s 11ms/step - loss: 0.6928 - accuracy: 0.5187 - val_loss: 0.6943 - val_accuracy: 0.4980
Epoch 58/120
313/313 [==============================] - 4s 11ms/step - loss: 0.6922 - accuracy: 0.5211 - val_loss: 0.6950 - val_accuracy: 0.5000
Epoch 59/120
313/313 [==============================] - 4s 11ms/step - loss: 0.6919 - accuracy: 0.5255 - val_loss: 0.6933 - val_accuracy: 0.4950
Epoch 60/120
313/313 [==============================] - 3s 10ms/step - loss: 0.6894 - accuracy: 0.5297 - val_loss: 0.6926 - val_accuracy: 0.4970
Epoch 61/120
313/313 [==============================] - 3s 10ms/step - loss: 0.6877 - accuracy: 0.5321 - val_loss: 0.6912 - val_accuracy: 0.5030
Epoch 62/120
313/313 [==============================] - 3s 10ms/step - loss: 0.6862 - accuracy: 0.5406 - val_loss: 0.6861 - val_accuracy: 0.5290
Epoch 63/120
313/313 [==============================] - 3s 11ms/step - loss: 0.6790 - accuracy: 0.5533 - val_loss: 0.6791 - val_accuracy: 0.551

313/313 [==============================] - 3s 11ms/step - loss: 0.3871 - accuracy: 0.8220 - val_loss: 0.4326 - val_accuracy: 0.7880
Epoch 114/120
313/313 [==============================] - 4s 11ms/step - loss: 0.3891 - accuracy: 0.8243 - val_loss: 0.4229 - val_accuracy: 0.7860
Epoch 115/120
313/313 [==============================] - 3s 9ms/step - loss: 0.3840 - accuracy: 0.8252 - val_loss: 0.4318 - val_accuracy: 0.7900
Epoch 116/120
313/313 [==============================] - 3s 10ms/step - loss: 0.3861 - accuracy: 0.8226 - val_loss: 0.4282 - val_accuracy: 0.7940
Epoch 117/120
313/313 [==============================] - 3s 10ms/step - loss: 0.3865 - accuracy: 0.8190 - val_loss: 0.4417 - val_accuracy: 0.7790
Epoch 118/120
313/313 [==============================] - 3s 10ms/step - loss: 0.3829 - accuracy: 0.8258 - val_loss: 0.4335 - val_accuracy: 0.7900
Epoch 119/120
313/313 [==============================] - 3s 10ms/step - loss: 0.3837 - accuracy: 0.8284 - val_loss: 0.4311 - val_accuracy: 

<p> On sauvgarde le modèle </p>

In [47]:
filename = 'chatbot.h5'
model.save(filename)

<p> On importe le modèle sauvgarder et on fait une prédiction </p>

In [83]:
model.load_weights(filename)
pred_results = model.predict(([inputs_test, queries_test]))

32/32 [==============================] - 0s 9ms/step


In [84]:
story =' '.join(word for word in test_data[6][0])
print(story)

Daniel went back to the kitchen . Mary grabbed the apple there . Daniel journeyed to the office . John went back to the office .


In [85]:
query = ' '.join(word for word in test_data[6][1])
print(query)

Is Daniel in the hallway ?


In [86]:
print("True Test Answer from Data is:",test_data[6][2])

True Test Answer from Data is: no


In [88]:
pred_results[6]

array([1.44307885e-10, 1.06817936e-10, 6.75345543e-11, 2.98000896e-06,
       8.78935258e-11, 9.70785050e-11, 9.26916391e-11, 1.03653995e-10,
       1.13779125e-10, 1.04337330e-10, 9.79919271e-11, 9.99997020e-01,
       8.81276996e-11, 1.22476709e-10, 7.50097345e-11, 1.31800959e-10,
       1.20896140e-10, 9.41382111e-11, 9.57071575e-11, 9.11585599e-11,
       1.04153214e-10, 7.88830390e-11, 1.11560990e-10, 9.43037246e-11,
       9.60505980e-11, 1.07073322e-10, 1.40758932e-10, 9.73044839e-11,
       5.60347949e-11, 9.40531403e-11, 9.44796186e-11, 9.19859189e-11,
       1.03219176e-10, 8.92293045e-11, 1.37223941e-10, 8.93181917e-11,
       1.15663951e-10, 1.20860863e-10], dtype=float32)

In [89]:
#Generate prediction from model
val_max = np.argmax(pred_results[6])

for key, val in tokenizer.word_index.items():
    if val == val_max:
        k = key

print("Predicted answer is: ", k)
print("Probability of certainty was: ", pred_results[6][val_max])

Predicted answer is:  no
Probability of certainty was:  0.999997


<p> On créé notre propre histoire et question. Attention, il faut utiliser seulement les mots  qu'il y a dans <code>vocabs</code> </p>

In [54]:
# Note the whitespace of the periods
my_story = "Erika left the bathroom . John dropped the football in the garden ."
my_story.split()

['John',
 'left',
 'the',
 'kitchen',
 '.',
 'Sandra',
 'dropped',
 'the',
 'football',
 'in',
 'the',
 'garden',
 '.']

In [55]:
my_question = "Is the football in the garden ?"

In [56]:
mydata = [(my_story.split(),my_question.split(),'yes')]

In [57]:
my_story,my_ques,my_ans = vectorize_stories(mydata)

In [58]:
pred_results = model.predict(([ my_story, my_ques]))

1/1 [==============================] - 0s 6ms/step


In [59]:
#Generate prediction from model
val_max = np.argmax(pred_results[0])

for key, val in tokenizer.word_index.items():
    if val == val_max:
        k = key

print("Predicted answer is: ", k)
print("Probability of certainty was: ", pred_results[0][val_max])

Predicted answer is:  yes
Probability of certainty was:  0.9977841
